In [21]:
import numpy as np
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)


#bring in dipoles and energies
dipoles = np.load("LiH_r_1.4_6311g_fci_Dipoles_200.npy")
energies = np.load("LiH_r_1.4_6311g_fci_Energies_200.npy")


print(energies)
#print(dipoles)

omega = 0.12086 

#lambda vectors 
lambda_vec = np.array([0.0, 0.0, 0.05])

m_n = 0 #num_photons

mu_n = 0 #electronic state

n_elec = 100
energies = energies[:n_elec]





[-8.01219476 -7.899962   -7.88593596 -7.86104863 -7.86104863 -7.84851087
 -7.84851087 -7.80693549 -7.78459936 -7.76702524 -7.76342939 -7.75827098
 -7.75827098 -7.75123644 -7.75123644 -7.72171149 -7.68303148 -7.53167595
 -7.46183148 -7.44561152 -7.44561152 -7.44122102 -7.41700688 -7.41700688
 -7.41499491 -7.41499491 -7.40506676 -7.40302238 -7.39081209 -7.38221021
 -7.38221021 -7.37219855 -7.37219855 -7.36897222 -7.36897222 -7.35609042
 -7.35609042 -7.33714534 -7.33343267 -7.32130747 -7.31448363 -7.31448363
 -7.306799   -7.306799   -7.30583178 -7.30363511 -7.29983049 -7.29983049
 -7.29710106 -7.28736609 -7.28736609 -7.27939388 -7.27254896 -7.26716866
 -7.26189291 -7.26189291 -7.26028229 -7.26028229 -7.24463063 -7.24419397
 -7.24419397 -7.24111323 -7.23094221 -7.22795844 -7.22795844 -7.2276023
 -7.2276023  -7.21883619 -7.21553431 -7.20683738 -7.20683738 -7.19644038
 -7.18180272 -7.17656251 -7.15921998 -7.15921998 -7.15824942 -7.15347534
 -7.15347534 -7.14947376 -7.14947376 -7.13886123 -7.

In [22]:

def build_d_matrix(lambda_vec, dipoles):

    d_matrix = np.einsum('k, ijk -> ij', lambda_vec, dipoles[:n_elec, : n_elec, :])

    #for when the matrix is only upper triangular
    d_matrix = d_matrix + d_matrix.T - np.diag(np.diag(d_matrix))

    return d_matrix
    #return np.tensordot(dipoles, lambda_vec, axes = 1)


d_matrix = build_d_matrix(lambda_vec, dipoles)

#d_matrix = d_matrix + d_matrix.T + np.diag(np.diag(d_matrix))
#d_matrix = d_matrix + d_matrix.T
print(d_matrix[0:4, 0:4])


def delta(i,j):
    if i ==j:
        return 1
    else:
        return 0



def calc_first_order_energy_correction(d_matrix, omega, mu_n):

    E_n_1 = 0
    for mu_l in range(0, len(energies)):
        E_n_1 += d_matrix[mu_n][mu_l]*d_matrix[mu_l][mu_n]

    E_n_1 = (1/2) * E_n_1

    return E_n_1




def calc_second_order_energy_correction(d_matrix, omega, mu_n, energies, m_n):
    E_n_2 = 0


    for mu_l in range(0, len(energies)):

            #for m_l in range(max(m_n-1,0), m_n+2):
            for m_l in range(max(0, m_n-1), m_n+2 ):

                if(mu_l == mu_n and m_l == m_n):
                    pass
                else:

                    sum_over_energy_states_gamma = 0
                    for gamma in range(0, len(energies)):
                        #numerator += d_matrix[gamma][mu_l]*d_matrix[mu_l][gamma]
                        sum_over_energy_states_gamma += (d_matrix[mu_l][gamma]*d_matrix[gamma][mu_n]  * delta(m_l, m_n) )

                    
                    E_n_2 += (((-(np.sqrt(omega/2))* d_matrix[mu_l][mu_n] * (np.sqrt(m_n +1) * delta(m_l, m_n + 1)  + np.sqrt(m_n) * delta(m_l, m_n-1)) ) + (0.5  * sum_over_energy_states_gamma))**2   )/ (energies[mu_n] + (m_n * omega) - energies[mu_l] - (m_l * omega))



    return E_n_2




def calc_third_order_energy_correction(d_matrix, omega, mu_n, energies, m_n):
    E_n_3 = 0

    for mu_p in range(0, len(energies)):

        for mu_q in range(0,len(energies)):
            
            for m_p in range(max(0, m_n-1), m_n+2):  

                for m_q in range(max(0,m_n-1), m_n+2):
                    
                    if((mu_p == mu_n and m_p == m_n) or (mu_q == mu_n and m_q == m_n)):
                        pass
                    else:
                        #do stuff
                        first_part_first_term = ((d_matrix[mu_n][mu_p])*(np.sqrt(m_p + 1) * delta(m_n, m_p+1) + np.sqrt(m_p) * delta(m_n, m_p-1)  ))


                        second_part_first_term = 0
                        for gamma in range(0, len(energies)):
                            second_part_first_term +=  d_matrix[mu_n][gamma]*d_matrix[gamma][mu_p] * delta(m_n, m_p)

                        first_term = ((-np.sqrt(omega/2)) *first_part_first_term ) + (0.5 * second_part_first_term)


                        first_part_second_term = ((d_matrix[mu_p][mu_q])*(np.sqrt(m_q + 1) * delta(m_p, m_q+ 1) + np.sqrt(m_q) * delta(m_p, m_q-1)  ))

                        second_part_second_term = 0
                        for gamma in range(0, len(energies)):
                            second_part_second_term +=  d_matrix[mu_p][gamma]*d_matrix[gamma][mu_q]* delta(m_p, m_q)

                        second_term = ((-np.sqrt(omega/2)) * first_part_second_term ) + (0.5 * second_part_second_term)


                        first_part_third_term = ((d_matrix[mu_q][mu_n])*(np.sqrt(mu_n + 1) * delta(m_q, m_n+1) + np.sqrt(mu_n) * delta(m_q, m_n-1)  ))

                        second_part_third_term = 0
                        for gamma in range(0, len(energies)):
                            second_part_third_term +=  d_matrix[mu_q][gamma]*d_matrix[gamma][mu_n] * delta(m_q, m_n)
                        third_term = ((-np.sqrt(omega/2)) * first_part_third_term) + (0.5 * second_part_third_term)


                        E_n_3 += (first_term*second_term*third_term) / ( ( energies[mu_n] + (m_n*omega) - energies[mu_p] - (m_p*omega)   ) *  (energies[mu_n] + (m_n*omega) - energies[mu_q] - (m_q*omega)  )   )


    
    #E_N_3 has a term that looks like E_n_1
    E_n_1 = 0
    for mu_l in range(0, len(energies)):
        E_n_1 += d_matrix[mu_n][mu_l]*d_matrix[mu_l][mu_n]

    E_n_1 = (1/2) * E_n_1


    #E_N_3 has a term that looks like E_n_2 except denominator is squared
    E_n_2_similar = 0


    for mu_l in range(0, len(energies)):

            #for m_l in range(max(m_n-1,0), m_n+2):
            for m_l in range(max(0, m_n-1), m_n+2 ):

                if(mu_l == mu_n and m_l == m_n):
                    pass
                else:

                    sum_over_energy_states_gamma = 0
                    for gamma in range(0, len(energies)):
                        #numerator += d_matrix[gamma][mu_l]*d_matrix[mu_l][gamma]
                        sum_over_energy_states_gamma += (d_matrix[mu_l][gamma]*d_matrix[gamma][mu_n] * delta(m_l, m_n))

                    
                    E_n_2_similar += (((-(np.sqrt(omega/2))* d_matrix[mu_l][mu_n] * (np.sqrt(m_n +1) * delta(m_l, m_n + 1)  + np.sqrt(m_n) * delta(m_l, m_n-1)) ) + (0.5  * sum_over_energy_states_gamma))**2   )/ (energies[mu_n] + (m_n * omega) - energies[mu_l] - (m_l * omega))**2


    return E_n_3 - (E_n_1 * E_n_2_similar)





def calc_energy_correction_to_first_order(mu_n, E_n_1):
    return energies[mu_n] + (omega * m_n) + E_n_1 

def calc_energy_correction_to_second_order(mu_n, E_n_1, E_n_2):
    return energies[mu_n] + (omega * m_n) + E_n_1 + E_n_2 

def calc_energy_correction_to_third_order(mu_n, E_n_1, E_n_2, E_n_3):
    return energies[mu_n] + (omega * m_n) + E_n_1 + E_n_2 + E_n_3




E_n_1 = calc_first_order_energy_correction(d_matrix, omega, mu_n)
print("E_n_1: ", E_n_1)

E_n_2 = calc_second_order_energy_correction(d_matrix, omega, mu_n, energies, m_n=m_n)
print("E-n_2: ", E_n_2)


E_n_3 = calc_third_order_energy_correction(d_matrix, omega, mu_n, energies, m_n=m_n)
print("E_n_3: ", E_n_3)



print("uncorrected_energy:" , energies[mu_n]+(omega * m_n))

print("energy corrected to first order: " , calc_energy_correction_to_first_order(mu_n, E_n_1))
print("energy corrected to second order: " , calc_energy_correction_to_second_order(mu_n, E_n_1, E_n_2))
print("energy corrected to third order: " , calc_energy_correction_to_third_order(mu_n, E_n_1, E_n_2, E_n_3))



[[-1.70754464e-01 -2.03971905e-15  4.50310795e-02  1.71938830e-15]
 [-2.03971905e-15  2.21149556e-02  1.25523199e-15  1.04908841e-15]
 [ 4.50310795e-02  1.25523199e-15  3.82760653e-02  4.65664274e-15]
 [ 1.71938830e-15  1.04908841e-15  4.65664274e-15 -6.16544172e-02]]
E_n_1:  0.01814216959354599
E-n_2:  -0.016406961676100194
E_n_3:  0.0014687973038413644
uncorrected_energy: -8.012194758182286
energy corrected to first order:  -7.9940525885887395
energy corrected to second order:  -8.010459550264839
energy corrected to third order:  -8.008990752960997


In [23]:
print(E_n_1)
print(E_n_2)
print(E_n_3)

print("uncorrected_energy:" , energies[mu_n]+(omega * m_n))

print("energy corrected to second order: " , calc_energy_correction_to_second_order(mu_n, E_n_1, E_n_2))
print("energy corrected to third order: " , calc_energy_correction_to_third_order(mu_n, E_n_1, E_n_2, E_n_3))



0.01814216959354599
-0.016406961676100194
0.0014687973038413644
uncorrected_energy: -8.012194758182286
energy corrected to second order:  -8.010459550264839
energy corrected to third order:  -8.008990752960997


In [17]:

#coherent state transformation
lambda_vec = np.array([0.0, 0.0, 0.05])
d_matrix = build_d_matrix(lambda_vec, dipoles)


#mu => (50,50,3)
#np.dot(mu[0,0,:], lambda_vec)
print(d_matrix[0:4, 0:4])
d_expectation_value = np.dot(dipoles[0,0,:], lambda_vec)
print(d_expectation_value)
#print(np.dot( d_expectation_value, np.ones((n_elec, n_elec))))

#print(np.subtract(d_matrix , np.dot( d_expectation_value, np.eye(n_elec, n_elec)))[0:4, 0:4])


def coherent_state_d_matrix():
    d_matrix = build_d_matrix(lambda_vec, dipoles)
    print(d_matrix[0:4, 0:4])
    print(np.dot(d_expectation_value, np.eye(n_elec, n_elec))[0:4, 0:4])
    d_matrix = d_matrix - np.dot(d_expectation_value, np.eye(n_elec, n_elec))
    return d_matrix


d_matrix = coherent_state_d_matrix()
print(d_matrix[0:4, 0:4])

E_n_1 = calc_first_order_energy_correction(d_matrix, omega, mu_n)

E_n_2 = calc_second_order_energy_correction(d_matrix, omega, mu_n, energies, m_n=m_n)

E_n_3 = calc_third_order_energy_correction(d_matrix, omega, mu_n, energies, m_n=m_n)


print("E_n_1: ", E_n_1)
print("E-n_2: ", E_n_2)
print("E-n_3: ", E_n_3)


print("uncorrected_energy:" , energies[mu_n]+(omega * m_n))
print("energy corrected to second order: " , calc_energy_correction_to_second_order(mu_n, E_n_1, E_n_2))
print("energy corrected to third order: " , calc_energy_correction_to_third_order(mu_n, E_n_1, E_n_2, E_n_3))



[[-1.70754464e-01 -2.03971905e-15  4.50310795e-02  1.71938830e-15]
 [-2.03971905e-15  2.21149556e-02  1.25523199e-15  1.04908841e-15]
 [ 4.50310795e-02  1.25523199e-15  3.82760653e-02  4.65664274e-15]
 [ 1.71938830e-15  1.04908841e-15  4.65664274e-15 -6.16544172e-02]]
-0.17075446429788352
[[-1.70754464e-01 -2.03971905e-15  4.50310795e-02  1.71938830e-15]
 [-2.03971905e-15  2.21149556e-02  1.25523199e-15  1.04908841e-15]
 [ 4.50310795e-02  1.25523199e-15  3.82760653e-02  4.65664274e-15]
 [ 1.71938830e-15  1.04908841e-15  4.65664274e-15 -6.16544172e-02]]
[[-0.17075446  0.          0.          0.        ]
 [ 0.         -0.17075446  0.          0.        ]
 [ 0.          0.         -0.17075446  0.        ]
 [ 0.          0.          0.         -0.17075446]]
[[ 0.00000000e+00 -2.03971905e-15  4.50310795e-02  1.71938830e-15]
 [-2.03971905e-15  1.92869420e-01  1.25523199e-15  1.04908841e-15]
 [ 4.50310795e-02  1.25523199e-15  2.09030530e-01  4.65664274e-15]
 [ 1.71938830e-15  1.04908841e-15  